In [1]:
from matplotlib import pyplot as plt
from mtcnn.mtcnn import MTCNN
import cv2
import time

from os import listdir
from os.path import isfile, join

import numpy as np

import imghdr

detector = MTCNN()

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [2]:
#IMG_PATH = "/home/image/sample/설현" # 30-60 구간에 이상한거 2개, 60-100 구간에도 1개 (바둑판배열인게 그럼)
IMG_PATH = "/home/jeinsong/example"
files = [join(IMG_PATH, f) for f in listdir(IMG_PATH) if isfile(join(IMG_PATH, f)) and imghdr.what(join(IMG_PATH, f))]
len(files), files[:5]
files[20]

'/home/jeinsong/example/test11.jpg'

## 주의사항
- skimage.io.imread() : RGB(RGBA)
- cv2.imread() : BGR(BGRA)
- detector는 인풋으로 RGB 이미지를 기대함
- image.shape ; (height, width , channel)

In [3]:
import uuid
def face_detection(file):
    image = cv2.imread(file) # bgr
    THRES = 500
    if (image.shape[1] > THRES):
        ratio = THRES/image.shape[1]
        image = cv2.resize(image, (THRES, int(image.shape[0] * ratio)))
    converted_image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # rgb
    faces = detector.detect_faces(converted_image) # rgb 
    
    # 썸네일 생성
    make_thumbnail_hub_main(image, faces)
    
    # 얼굴 인식 부분 표시
    for f in faces:
        cv2.rectangle(
            image
            , (f["box"][0], f["box"][1])
            , (f["box"][0] + f["box"][2], f["box"][1] + f["box"][3])
            , (0,0,255), 2
        );
        cv2.putText(image
                    , "%dx%d" % (f['box'][2], f['box'][3])
                    ,(f['box'][0] + 10, f['box'][1] + 20)
                    , cv2.FONT_HERSHEY_SIMPLEX
                    , 0.6, (0,255,0), 2
                   )
        '''
        cv2.putText(image
                    , "(%.2f)" % (f['confidence'])
                    ,(f["box"][0] + f["box"][2], f["box"][1] + f["box"][3])
                    , cv2.FONT_HERSHEY_SIMPLEX
                    , 0.6, (0,255,0), 2
                   )'''
        
    ext = imghdr.what(file)
    filename = str(uuid.uuid4()) + "." + ext
    cv2.imwrite("./tmp2/_%s" %(filename), image) # bgr -> rgb
    cv2.imwrite("./tmp2/%s" %(filename), cv2.imread(file))
    return filename

### make_thumbnail 흐름
![makethumbnail](makethumbnail2.png)

In [4]:
"""
    지금은 hub zum main 썸네일 크기 (270, 200) 으로 하지만,
    파라미터화 해서 모두 대응 가능하게 바꿀 것
    
    Input: 이미지(가로길이 최대 500 가정), 얼굴 배열
"""
def make_thumbnail_hub_main(image, faces):
    if len(faces) == 0:
        return
    
    h = image.shape[0]
    w = image.shape[1]
    
    """
        너무 작은 얼굴 필터링
        - 가장 큰 얼굴의 절반 정도 이하인 얼굴들 제거
        - (긴 이미지의 경우 맨 위에 작은 얼굴들이 모여있다면 문제가 될 수 있긴함)
    """
    max_face_height = max([f["box"][3] for f in faces])
    faces_filtered = [f for f in faces if f["box"][2] > max_face_height * 0.3 ] 
    faces = faces_filtered if len(faces_filtered) > 0 else faces
    
    print("faces len %d, max f h %d" %(len(faces), max_face_height))
    """
        세로로 긴 이미지이면서 "여러장이 붙어서 만들어 진 이미지"인지 판단
        - 여러장이 붙어서 만들어진 이미지의 경우 '무조건' 맨위에 자른다 (crop area 의 높이 만큼만)
    """
    if (w * 1.5 < h):
        print ("LONG VERTICAL IMAGE") 
        if is_vertical_long_images(faces ,h, 4):
            print ("multiple images attached")
            faces = [f for f in faces if f["box"][1] < 200] # 200은 crop area 의 높이
        
    len_faces = len(faces)
    """
        1. 얼굴이 없을 경우
            - 중앙을 중심으로 최대 사이즈에 맞춰서
        2. 1개인 경우
        3. 2개인 경우 or 3개인 경우
            - 2~3 명을 다 묶어서 크로핑
        4. 4명 이상인 경우 
            - (가운데 인물 위주 크로핑)
    """
    ca = (0,0, 100, 100)
    if len_faces == 0:
        pass
    elif len_faces == 1:
        ca = get_crop_area_hub_main(image, faces[0]["box"])
    elif len_faces == 2 or len_faces == 3:
        ca = get_crop_area_hub_main(image, merge_faces_as_box(faces))
    else:
        cf = find_central_face(faces)
        ca = get_crop_area_hub_main(image, cf["box"])
                                        
    cv2.rectangle(
        image
        , (ca[0], ca[1])
        , (ca[0] + ca[2], ca[1] + ca[3])
        , (255,0,0), 2
    );

"""
    이미지가 vertical & long 일 경우
    이미지를 세로로 num_div 등분해서
    각 그룹에 등장하는 모양을 보고 판단
"""
def is_vertical_long_images (faces, h, num_div):
    cnt_faces = [0] * num_div
    unit_h = h / num_div
    print("unit_h %f" %(unit_h))
    for f in faces:
        f_y = f["box"][1]
        f_w = f["box"][2]
        cnt_faces[int(f_y / unit_h)] += 1
    
    #gcnt = sum([c for c in cnt_faces if c > 0]) # 사실 단순합 2가 아닌 서로 떨어져있으면 더 확신을 가져도 됨
    #return gcnt > 1
    return (cnt_faces[0] > 0 or cnt_faces[num_div - 1] > 0)
    

"""
    인물이 여러명(2~3명)일 경우, get_crop_area 함수를 통해 크로핑 영역을 선정함에 있어서
    파라미터를 하나의 박스로 전달해야 하기 때문에, 여러 인물 박스를 하나로 합칠 필요가 있다.
    Input : faces
    Output : (box_x, box_y, box_width, box_height)
"""
def merge_faces_as_box(faces):
    # most top y pos
    mty = 9999
    # most left x pos
    mlx = 9999
    # most bottom y pos
    mby = -1
    # most right x pos
    mrx = -1
    
    for f in faces:
        mty = min(mty, f["box"][1])
        mlx = min(mlx, f["box"][0])
        mby = max(mby, f["box"][1] + f["box"][3])
        mrx = max(mrx, f["box"][0] + f["box"][2])
    
    return (mlx, mty, mrx - mlx, mby - mty)
        
def find_central_face(faces):
    # 일단 가로 위치 기준으로 정렬해봄
    faces = sorted(faces, key=lambda face: face['box'][0])
    # 살짝 큰 것에 가중치를 줄 필요가 있음..
    return faces[int(len(faces)/2)]
    
"""
    hub main cropping (270 x 200) 
    box 영역을 포함하는 cropping 영역 구하기
    box : (x , y, width, height) mtcnn's format
    
    인물이 여러명일 경우 하나의 box로 묶어서 전달됨.
"""
def get_crop_area_hub_main(image, box):
    # crop area
    c_h = 200
    c_w = 270
    
    # image's width & height
    # cv image's shape: (height, width, channels)
    h = image.shape[0]
    w = image.shape[1]
    
    if c_w > w and c_h > h:
        print ("CROPPING AREA가 이미지보다 큽니다")
        # 이경우 ca 를 같은 비율로 줄일 필요가 있음
        return (0,0,50,50)
    
    # box's width & height
    b_x = box[0]
    b_y = box[1]
    b_w = box[2]
    b_h = box[3]
    
    if b_w > c_w or b_h > c_h:
        print ("BOX가 CROPPING AREA보다 큽니다")
        # ca를 같은 비율로 늘릴 필요
        c_h = int(c_h * 1.5)
        c_w = int(c_w * 1.5)
    
    """
        box 모서리를 기준으로 크롭을 위한 image 안에서의 공간이 여유로운지 판단해보기
        box 위/아래/왼쪽/오른쪽 으로 확장되야할 길이를 margin_h, margin_y 로 잡음
        (margin_h + b_h + margin_h) ==  crop area's height
        (margin_w + b_w + margin_w) = crop area's width
    """
    
    # box 가 crop area 보다 작다고 가정
    margin_h = (c_h - b_h) / 2
    margin_w = (c_w - b_w) / 2
    
    # cropping area left-top position
    c_x = -1
    c_y = -1
    
    """
        cropping 영역이 이미지의 안쪽에 넉넉히 위치해 있을 경우엔 쉽지만
        애매하게 경계에 걸쳐있는 경우엔 처리가 필요
    """
    if b_y - margin_h < 0: # 위로 공간 모자를 때
        c_y = 0
    if b_y + b_h + margin_h > h: # 아래로 공간 모자를 때
        c_y = h - c_h
    if b_x - margin_w < 0: # 왼쪽으로 공간 모자를 때
        c_x = 0
    if b_x + b_w + margin_w > w: # 오른쪽으로 공간 모자를 때
        c_x = w - c_w
        
    """
        예외처리에 걸리지 않은 경우
    """
    if c_x == -1:
        c_x = b_x - margin_w
    if c_y == -1:
        c_y = b_y - margin_h

    return (int(c_x), int(c_y), c_w, c_h)

In [5]:
fd_files = []
for idx, f in enumerate(files):
    print ("idx : ", idx)
    fd = face_detection(f)
    fd_files.append(fd)

idx :  0
faces len 2, max f h 169
LONG VERTICAL IMAGE
unit_h 206.500000
multiple images attached
idx :  1
faces len 1, max f h 133
idx :  2
faces len 1, max f h 118
idx :  3
idx :  4
faces len 2, max f h 58
idx :  5
faces len 10, max f h 55
idx :  6
faces len 3, max f h 165
BOX가 CROPPING AREA보다 큽니다
idx :  7
faces len 3, max f h 78
BOX가 CROPPING AREA보다 큽니다
idx :  8
faces len 2, max f h 33
idx :  9
faces len 6, max f h 85
LONG VERTICAL IMAGE
unit_h 342.750000
multiple images attached
idx :  10
faces len 1, max f h 135
LONG VERTICAL IMAGE
unit_h 187.750000
multiple images attached
idx :  11
faces len 2, max f h 187
LONG VERTICAL IMAGE
unit_h 222.000000
BOX가 CROPPING AREA보다 큽니다
idx :  12
faces len 3, max f h 87
BOX가 CROPPING AREA보다 큽니다
idx :  13
faces len 4, max f h 84
LONG VERTICAL IMAGE
unit_h 300.000000
multiple images attached
idx :  14
faces len 3, max f h 143
BOX가 CROPPING AREA보다 큽니다
idx :  15
faces len 1, max f h 144
idx :  16
faces len 3, max f h 137
LONG VERTICAL IMAGE
unit_h 214.

# 썸네일 기본 사이즈
- 허브줌
  - 메인: 270 x 200
  - 콘텐츠 뷰 우측: 83 x 64
- 뉴스줌
  - 뉴스 기사 최신 목록: 100 x 63
  - 기사박스 하단 데이블: 130 x 88
  - 포토 슬라이드: 81 x 81

In [6]:
from jinja2 import Template
from IPython.core.display import HTML

t = Template("""
<table>
    {% for idx in range(files | length) %}
    <tr>
        <td> {{ idx }} </td>
        <td>
            <img  style="float:left; width:500px" src="./tmp2/{{ files[idx] }}">
        </td>
        <td>
            <img  style="float:left; width:500px" src="./tmp2/_{{ files[idx] }}">
        </td>
    </tr>
    {% endfor %}
</table>
""")
HTML(t.render(files=fd_files))

0,,
1,,
2,,
3,,
4,,
5,,
6,,
7,,
8,,
9,,
10,,


## Rule 정리중

- 여러 얼굴 -> 가운데 부터 출발
- 여러 얼굴 -> 이미지 비율보다 너무 작은 얼굴 제외 후
- 모두 없어진다면 중앙 자르기
- 인물 인식 갯수 1,2,3개, 4개 이상
- 500 기준 30 px 이하의 얼굴은 제거 (4개 이상의 경우만?)
- 세로가 가로의 1.5 배 이상인 사진은 가장 위 얼굴만
- 다 없어지거나 얼굴 인식 x 이미지 경우 세로 이미지냐 가로이미지냐에 다라 다르게처리
- ...
- 문제 있는 경우 url을 통해 아무 처리 안한 이미지를 던져주기
- 인물이 많으면 그냥 가운데 사람 중심으로 자르면됨(요주인물은 가운데 있잖아)

### 참고
- 허브줌
  - 메인: 270 x 200
  - 콘텐츠 뷰 우측: 83 x 64
- 뉴스줌
  - 뉴스 기사 최신 목록: 100 x 63
  - 기사박스 하단 데이블: 130 x 88
  - 포토 슬라이드: 81 x 81